<a href="https://colab.research.google.com/github/luiscunhacsc/curso_dl_session01/blob/main/ISMT_DL_Session01_ExC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Célula 1: Introdução
Neste exemplo, vamos explorar como usar modelos pré-treinados para classificação de imagens. O conjunto de dados que utilizaremos é o CIFAR-100, que consiste em 60.000 imagens coloridas de 32x32 pixels, distribuídas por 100 classes, com 600 imagens por classe.

Usaremos um modelo pré-treinado, faremos ajuste fino e utilizaremos técnicas como data augmentation e regularização.

Vamos começar importando as bibliotecas necessárias.

In [ ]:
# Célula 2: Importar bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

# Célula 3: Carregar o conjunto de dados CIFAR-100
Vamos carregar o conjunto de dados CIFAR-100 e fazer uma normalização das imagens.

In [ ]:
# Célula 4: Carregar e normalizar os dados
(train_images, train_labels), (test_images, test_labels) = cifar100.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
train_labels = to_categorical(train_labels, 100)
test_labels = to_categorical(test_labels, 100)

# Célula 5: Usando um modelo pré-treinado
Vamos utilizar o modelo ResNet50 que foi pré-treinado no conjunto de dados ImageNet. Inicialmente, usaremos o modelo diretamente para fazer previsões.

In [ ]:
# Célula 6: Utilizar o modelo pré-treinado
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Congelar todas as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Adicionar camadas personalizadas
x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
outputs = layers.Dense(100, activation='softmax')(x)

# Construir o modelo
model = Model(inputs=base_model.input, outputs=outputs)

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history_pretrained = model.fit(train_images, train_labels, batch_size=128, epochs=5, validation_data=(test_images, test_labels))

# Célula 7: Ajuste fino do modelo
Agora, vamos descongelar algumas das camadas do modelo base e realizar ajuste fino.

In [ ]:
# Célula 8: Descongelar camadas e realizar ajuste fino
for layer in base_model.layers[-4:]:
    layer.trainable = True

# Compilar o modelo
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history_fine_tuning = model.fit(train_images, train_labels, batch_size=128, epochs=5, validation_data=(test_images, test_labels))

# Célula 9: Data Augmentation
Agora vamos utilizar a técnica de data augmentation para melhorar o desempenho do nosso modelo. Esta técnica consiste em gerar novas imagens a partir das originais, aplicando transformações como rotação, zoom, inversão, etc.

In [ ]:
# Célula 10: Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_augmentation = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Treinar o modelo com data augmentation
history_with_augmentation = model.fit(data_augmentation.flow(train_images, train_labels, batch_size=128), epochs=5, validation_data=(test_images, test_labels))

# Célula 11: Avaliar e comparar o desempenho do modelo
Vamos avaliar o desempenho dos modelos e comparar como eles se saíram com diferentes técnicas.

In [ ]:
# Célula 12: Plotar gráficos de desempenho
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.plot(history_pretrained.history['val_accuracy'], label='Pré-treinado')
plt.plot(history_fine_tuning.history['val_accuracy'], label='Ajuste fino')
plt.plot(history_with_augmentation.history['val_accuracy'], label='Data Augmentation')
plt.legend()
plt.xlabel('Épocas')
plt.ylabel('Acurácia de Validação')
plt.title('Desempenho do Modelo')

plt.subplot(1, 2, 2)
plt.plot(history_pretrained.history['val_loss'], label='Pré-treinado')
plt.plot(history_fine_tuning.history['val_loss'], label='Ajuste fino')
plt.plot(history_with_augmentation.history['val_loss'], label='Data Augmentation')
plt.legend()
plt.xlabel('Épocas')
plt.ylabel('Perda de Validação')
plt.title('Desempenho do Modelo')

plt.show()

# Célula 13: Conclusão
Neste exemplo, explorámos como podemos utilizar modelos pré-treinados para classificação de imagens utilizando o conjunto de dados CIFAR-100.

Começámos utilizando o modelo diretamente, depois fizemos ajuste fino e finalmente utilizámos data augmentation.

É importante notar que escolher as melhores técnicas e hiperparâmetros pode depender do conjunto de dados e do problema específico. Experimentar e avaliar diferentes abordagens é uma parte importante do processo de desenvolvimento de modelos de deep learning.